As computing hardware manufacturers add more cores to computer processors and multi-core processors becoming the norm even with personal computers and laptops, creating parallelizable code becomes very important. Python introduced the `multiprocessing` module to let us write parallel processing code to leverage the capabilities of multi-core CPUs.

**Note** - Improvement due to parallel processing is mostly noticable only if the tasks at hand are cpu-bound where the majority of the time is spent in the CPU in contrast to I/O bound tasks, where most of the time is spent reading and writing data from the disk.

##### Multithreading vs Multiprocessing
With multithreading, jobs are submitted to different threads, and it will behave more like subtasks of a single process. The different threads run in the same shared memory space, so care needs to be taken so that different threads do not write in the same memory location. In Python multithreading is not true parallelization, since it does not bypass the Global Interpreter Lock and the parallelization is achieved using a time slicing mechanism. So the multiple threads will run on a single core of the processor.

With multiprocessing, each process has its own separate memory, so memory management is easier. But inter-process communication will be costlier because every object that is passed between processes needs to be pickled and unpickled. Also spawning a new process can be costlier than spawning a new thread. Both these overheads need to be taken into account while contemplating multiprocessing vs multithreading. Unlike multithreading, in multiprocessing the different processes runs on different cores of the processor, so it effectively bypasses Python's Global Interpreter Lock, hence achieving true parallelization. So if you want to max out the resources in a multi-core machine then using the `multiprocessing` module is the way to go.

We will try to compare and benchmark multithreading and multiprocessing in different scenarios with a few practise probelms in the next notebook.

#### Important classes within the multiprocessing module - **Process, Queue, Pool and Lock**

In [2]:
import os
import sys
import time
import multiprocessing

###### Find out the number of cpu cores in your system

In [2]:
print("Number of cores = ", multiprocessing.cpu_count())

Number of cores =  4


##### Process
Process is a class that allows the parent process to start a new process, allows it to run code but have the parent process control its execution.

It has two main functions `start()` and `join()`. A Process object is instantiated by passing it a target function which the process will execute, in the example below the target function is `print_work()`. The process will only start executing when `start()` function is called. After completion the Process will return the result and remain idle. It only terminates by the use of the `join()` function.

In [6]:
def print_work(num):
    print("I am worker number {} and my process id is {}".format(num, os.getpid()))

workers = [multiprocessing.Process(target=print_work, args=(i,)) for i in range(5)]
for w in workers:
    w.start()

for w in workers:
    w.join()

print("All workers have finished working")

I am worker number 0 and my process id is 55367
I am worker number 1 and my process id is 55368
I am worker number 2 and my process id is 55369
I am worker number 3 and my process id is 55370
I am worker number 4 and my process id is 55371
All workers have finished working


Worker processes, instead of target functions can also accept target class to perform their task.

In [19]:
class Worker(multiprocessing.Process):
    def run(self):
        print("I am a process with id number {}".format(os.getpid()))
        return

workers = [Worker() for i in range(5)]
for w in workers:
    w.start()

for w in workers:
    w.join()

print("All workers have finished working")

I am a process with id number 56067
I am a process with id number 56068
I am a process with id number 56069
I am a process with id number 56070
I am a process with id number 56071
All workers have finished working


##### Queue
Like conventional queues, the Queue class is a first-in-first-out data structure provided by the multiprocessing module. They can store any object that can be pickled, and are used for inter-process communication. The Queue data structure provided by the multiprocessing module is thread-safe.

In [6]:
# Take a list of numbers and determine if they are prime or not.

# the function that each worker will run
def check_prime(qin, qout):
    while True:
        if qin.empty():
            break
        num = qin.get()
        time.sleep(0.1)
        is_prime = True
        for i in range(2,num):
            if num % i == 0:
                is_prime=False
                break
        qout.put((os.getpid(), num, is_prime))

tic = time.time()
# the input list of numbers and add it to an input queue
numbers = [222377,3334,2212,3345,23441,454577,341737,565633]
in_qu = multiprocessing.Queue()
for n in numbers:
    in_qu.put(n)
    
# the output queue into which each worker will put the output
out_qu = multiprocessing.Queue()

# use 5 workers to work on the list of numbers
workers = [multiprocessing.Process(target=check_prime, args=(in_qu, out_qu)) for i in range(5)]
for w in workers:
    w.start()

for w in workers:
    w.join()

print("All workers have finished working")
while not out_qu.empty():
    res = out_qu.get()
    print("Worker with process id {} says {} is {} prime".format(res[0], res[1], res[2]))
toc = time.time()
print("Total time taken: ", toc-tic)

All workers have finished working
Worker with process id 1022 says 222377 is False prime
Worker with process id 1023 says 3334 is False prime
Worker with process id 1024 says 2212 is False prime
Worker with process id 1025 says 3345 is False prime
Worker with process id 1026 says 23441 is False prime
Worker with process id 1023 says 341737 is False prime
Worker with process id 1024 says 565633 is False prime
Worker with process id 1022 says 454577 is True prime
Total time taken:  0.2512199878692627


##### Pool
Pool class represents a pool of worker processes. This abstraction allows tasks and work to be offloaded  on to the pool of workers. 

In [8]:
# We will accomplish the same task as checking for prime numbers but use worker pool to do it
# the function that each worker will run
def check_prime(num):
    time.sleep(0.1)
    is_prime = True
    for i in range(2,num):
        if num % i == 0:
            is_prime=False
            break
    print("Process {} says that {} is {} prime".format(os.getpid(), num, is_prime))

tic = time.time()
# the input list of numbers and add it to an input queue
numbers = [222377,3334,2212,3345,23441,454577,341737,565633]

# use 5 workers to work on the list of numbers
worker_pool = multiprocessing.Pool(5)
worker_pool.map(check_prime, numbers)

worker_pool.close()
worker_pool.join()

print("All workers have finished working")
toc = time.time()
print("Total time taken: ", toc-tic)

Process 2833 says that 3334 is False prime
Process 2832 says that 222377 is False prime
Process 2835 says that 3345 is False prime
Process 2836 says that 23441 is False prime
Process 2834 says that 2212 is False prime
Process 2835 says that 341737 is False prime
Process 2832 says that 565633 is False prime
Process 2833 says that 454577 is True prime
All workers have finished working
Total time taken:  0.3429441452026367


Other related functions in the Pool class is -
 - `imap`
 - `imap_unordered`
 - `starmap`
 - `starmap_async`
 - `map_async`
 - `apply_async`
 
<...>

##### Lock
<...>